In [1]:
%load_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import logging
logging.basicConfig(level=logging.INFO)

import pyrosetta
import pyrosetta.distributed.io as io
import pyrosetta.distributed.packed_pose as packed_pose
import pyrosetta.distributed.tasks.rosetta_scripts as rosetta_scripts
import pyrosetta.distributed.tasks.score as score

import pandas as pd
import pandas

import numpy as np

In [2]:
print(pyrosetta.version())

PyRosetta-4 2017 [Rosetta devel 2018.28.post.dev+1147.v2018.34dev60360.14e0a7acf8e14e0a7acf8eca980ba28bb1767f41fb5b7f52343 2018-08-21T11:28:04] retrieved from: ssh://git@github.com/RosettaCommons/main
(C) Copyright Rosetta Commons Member Institutions.
Created in JHU by Sergey Lyskov and PyRosetta Team.



In [3]:
import sys; sys.path.insert(0, "/home/ajasja/projects/Stable_Heterodimers")
import utils as u

In [4]:
ON_CLUSTER=False

In [5]:
flags = """
-beta \
-in:file:fullatom \
-multi_cool_annealer 10 \
-renumber_pdb 1 \
-overwrite 1 \
-out:pdb \
-out::file::pdb_comments \
-run:preserve_header \
"""

In [6]:
if not ON_CLUSTER:
    pyrosetta.init(flags)
else:
    cluster = u.init_env(flags, 10)

INFO:rosetta:Found rosetta database at: /home/ajasja/.conda/envs/pyro_scripts/lib/python3.6/site-packages/pyrosetta/database; using it....
INFO:rosetta:PyRosetta-4 2017 [Rosetta devel 2018.28.post.dev+1147.v2018.34dev60360.14e0a7acf8e14e0a7acf8eca980ba28bb1767f41fb5b7f52343 2018-08-21T11:28:04] retrieved from: ssh://git@github.com/RosettaCommons/main
(C) Copyright Rosetta Commons Member Institutions.
Created in JHU by Sergey Lyskov and PyRosetta Team.

INFO:rosetta:core.init: Checking for fconfig files in pwd and ./rosetta/flags
INFO:rosetta:core.init.score_function_corrections: [ WARNING ] Flag -beta_nov16 is set but -weights are also specified.  Not changing input weights file!
INFO:rosetta:core.init: Rosetta version: 2018.28.post.dev+1147.v2018.34dev60360.14e0a7acf8e 14e0a7acf8eca980ba28bb1767f41fb5b7f52343 ssh://git@github.com/RosettaCommons/main 2018-08-21T11:28:04
INFO:rosetta:core.init: command: PyRosetta -beta -in:file:fullatom -multi_cool_annealer 10 -renumber_pdb 1 -overwrite

In [7]:
def lines_grep(pattern, lines):
    import re
    reg = re.compile(pattern)
    return [l for l in lines if reg.match(l)]

def print_sele(scores, name, color=None, remove_chains=False):
    sele = scores[name+"_pymol_selection"]
    sele=sele.replace("rosetta_sele", name)
    if remove_chains:
        sele=sele.replace("chain A and", "").replace("chain B and", "")
    print(sele)
    if not color is None:
        print(f"color {color}, {name}")
    

In [8]:
#Pyrosetta unsupported BuriedUnsatHbonds2 InterfaceHoles CavityVolume
pack_protocol = u.read_file("truncator/xml/80_test_selectors.xml")
print(lines_grep(".*Neighborhood.*", pack_protocol.split("\n")))
task_design = rosetta_scripts.SingleoutputRosettaScriptsTask(pack_protocol)
# Syntax check via setup
logging.basicConfig(level=logging.WARN)
task_design.setup()
raw_input_pose = io.pose_from_file(filename="data/ZCON_37_N3_C1_2.pdb")
output_pose = task_design(raw_input_pose)
output_pose.scores;

INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: Generating XML Schema for rosetta_scripts...


['    <Neighborhood name="interface_chA" selector="chainB" distance="8.0"/>', '    <Neighborhood name="interface_chB" selector="chainA" distance="8.0"/>']


INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: ...done
INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: Initializing schema validator...
INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: ...done
INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: Validating input script...
INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: ...done
INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: Parsed script:
<ROSETTASCRIPTS>
	<SCOREFXNS>
		<ScoreFunction name="NOV15" weights="beta">
			<Reweight scoretype="coordinate_constraint" weight="1"/>
		</ScoreFunction>
		<ScoreFunction name="VDW" weights="empty">
			<Reweight scoretype="fa_atr" weight="1.0"/>
		</ScoreFunction>
		<ScoreFunction name="TotalHydrophobic" weights="/home/fseeger/PPI_Benchmark/Design_Scripts/total_hydrophobic_weights.wts"/>
	</SCOREFXNS>
	<RESIDUE_SELECTORS>
		<Layer core_cutoff="3.6" name="core" select_core="true" surface_cutoff="2.0" use_sidechain_neighbors="true

In [16]:
print("color deeppurple, all")
print_sele(output_pose.scores, "surf", "yellow")
print_sele(output_pose.scores, "bndr", "orange")
print_sele(output_pose.scores, "core", "red")


color deeppurple, all
select surf, (chain A and resid 2,3,4,7,8,10,11,14,15,18,21,25,28) or (chain B and resid 29,30,33,36,37,40,43,44,47,50,51,54,55,57,59,62,63,66,69,70,73,76,77,80,83,84,86,87,89,90,91,94,95,97,98,101,102,104,105,108,109,111,112,115,116,119)
color yellow, surf
select bndr, (chain A and resid 1,17,22,24) or (chain B and resid 32,34,39,41,46,48,53,56,58,65,72,79,92,93,118)
color orange, bndr
select core, (chain A and resid 5,6,9,12,13,16,19,20,23,26,27) or (chain B and resid 31,35,38,42,45,49,52,60,61,64,67,68,71,74,75,78,81,82,85,88,96,99,100,103,106,107,110,113,114,117)
color red, core


In [17]:
print("color deeppurple, all")
print_sele(output_pose.scores, "surf_SASA", "yellow")
print_sele(output_pose.scores, "bndr_SASA", "orange")
print_sele(output_pose.scores, "core_SASA", "red")

color deeppurple, all
select surf_SASA, (chain A and resid 1,2,3,4,7,8,11,14,15,18,21,22,25,28) or (chain B and resid 29,30,33,34,36,37,40,43,44,47,48,50,51,54,55,56,57,59,62,63,66,69,70,73,76,77,80,83,84,85,87,88,89,90,91,92,94,95,97,98,101,102,104,105,108,109,111,112,115,116,118,119)
color yellow, surf_SASA
select bndr_SASA, (chain A and resid 10,24,26) or (chain B and resid 32,39,41,58,67,74,81,117)
color orange, bndr_SASA
select core_SASA, (chain A and resid 5,6,9,12,13,16,17,19,20,23,27) or (chain B and resid 31,35,38,42,45,46,49,52,53,60,61,64,65,68,71,72,75,78,79,82,86,93,96,99,100,103,106,107,110,113,114)
color red, core_SASA


In [18]:

print("color deeppurple, all")
print_sele(output_pose.scores, "interface_by_vec", "orange")
print_sele(output_pose.scores, "core_and_interface_by_vec", "red")

color deeppurple, all
select interface_by_vec, (chain A and resid 1,2,3,5,6,7,8,9,10,12,13,15,16,17,19,20,21,22,23,24,26,27,28) or (chain B and resid 31,35,38,42,45,49,56,58,60,61,63,64,67,68,70,71,74,75,77,78,81,82,85,88,92,95,96,99,100,102,103,106,107,109,110,113,114,116,117)
color orange, interface_by_vec
select core_and_interface_by_vec, (chain A and resid 5,6,9,12,13,16,19,20,23,26,27) or (chain B and resid 31,35,38,42,45,49,60,61,64,67,68,71,74,75,78,81,82,85,88,96,99,100,103,106,107,110,113,114,117)
color red, core_and_interface_by_vec


In [19]:
print("color deeppurple, all")
print_sele(output_pose.scores, "AB_interface", "orange")
print_sele(output_pose.scores, "core_and_interface", "red")

color deeppurple, all
select AB_interface, (chain A and resid 1,2,3,5,6,7,8,9,10,12,13,15,16,17,19,20,21,22,23,24,26,27,28) or (chain B and resid 35,56,58,60,61,63,64,67,68,70,71,74,75,77,78,81,82,85,88,92,95,96,99,100,102,103,106,107,109,110,113,114,116,117)
color orange, AB_interface
select core_and_interface, (chain A and resid 5,6,9,12,13,16,19,20,23,26,27) or (chain B and resid 35,60,61,64,67,68,71,74,75,78,81,82,85,88,96,99,100,103,106,107,110,113,114,117)
color red, core_and_interface


In [20]:
u.write_file("test.pdb", io.to_pdbstring(output_pose))


In [14]:
pwd

'/home/ajasja/projects/truncator'

In [15]:
score_file = "test.sc" 
struct_name = "data/ZCON_37_N3_C1_2.pdb"
!/software/rosetta/latest/bin/rosetta_scripts -parser:protocol truncator/xml/80_test_selectors.xml -s {struct_name} \
    -out:file:scorefile  {score_file} \
    -print_pymol_selection 0 \
    -beta \
    -in:file:fullatom \
    -renumber_pdb 1 \
    -overwrite \
    -out:no_nstruct_label \
    -out:file:scorefile_format json > /dev/null
      

In [37]:
                          
scores = pd.read_json(score_file, lines=True).iloc[-1].to_dict()

In [47]:
print("color deeppurple, all")
print_sele(scores, "chainA", "orange")
print_sele(scores, "chainB", "red")

color deeppurple, all
select chainA, ( resid 92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119)
color orange, chainA
select chainB, ( resid 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91)
color red, chainB


In [48]:
print("color deeppurple, all")
print_sele(scores, "interface_by_vec", "orange")
print_sele(scores, "core_and_interface_by_vec", "red")

color deeppurple, all
select interface_by_vec, ( resid 92,93,94,96,97,98,99,100,101,103,104,106,107,108,110,111,112,113,114,115,117,118,119) or ( resid 3,7,10,14,17,21,28,30,32,33,35,36,39,40,42,43,46,47,49,50,53,54,57,60,64,67,68,71,72,74,75,78,79,81,82,85,86,88,89)
color orange, interface_by_vec
select core_and_interface_by_vec, ( resid 96,97,100,103,104,107,110,111,114,117,118) or ( resid 3,7,10,14,17,21,32,33,36,39,40,43,46,47,50,53,54,57,60,68,71,72,75,78,79,82,85,86,89)
color red, core_and_interface_by_vec
